In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np

# setup dev
device = device = torch.device("cuda:1")
print("device is", device)

# setup net
model = torchvision.models.vgg19(pretrained=True).to(device)
print("model is ", model)

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport deploy.dataloader
%aimport deploy.misc
%aimport methods.grad_cam
%aimport methods.grad_based
%aimport methods.occlusion

In [ ]:
from torchvision.transforms import ToTensor, RandomCrop, CenterCrop, Resize
from deploy.dataloader import *

def transform(img):
    """ shift color axes to back """
    return np.swapaxes(np.swapaxes(img, 0, 2), 0, 1)

def prepare(img):
    img = Resize(256)(img)
    #img = RandomCrop((224,224))(img)
    img = CenterCrop((224,224))(img)
    img = ToTensor()(img)
    img = img.unsqueeze(0)
    img = img.to(device)
    return img

# setup data
filename = '../data/imagenet_full/imagenet.hdf5'
show_loader = ImageNetDataset(filename, train=False)
dct = None
with open('../data/imagenet_full/dict.txt', encoding='utf-8') as data_file:
    dct = yaml.load(data_file.read())


In [ ]:
from methods.grad_based import *
from methods.grad_cam import *
from methods.occlusion import *
from deploy.misc import *

layer = find_layer(model.features, "34")
gc = GradCAM(model=model, device=device, layer=layer)

viz = Overlay()

methods = []
methods.append(gc)
#methods.append(Occlusion(model=model, size=20, stride=10))
#methods.append(Occlusion(model=model, size=20, stride=10, patch_type="color", patch_value=0.0))
#methods.append(Occlusion(model=model, size=25, stride=2, patch_type="inv"))
#methods.append(Occlusion(model=model, size=10, patch_type="inv"))
#methods.append(SmoothGrad(model=model, steps=1,std=0)) # = gradients
#methods.append(SmoothGrad(model=model, steps=30,std=0.2))
#methods.append(SmoothGrad(model=model, steps=1,std=0, cc_transform="mean")) # gradients
#methods.append(SmoothGrad(model=model, std=0, times_input=True))
#methods.append(SmoothGrad(model=model, times_input=True))
#methods.append(IntegratedGradients(model=model, steps=50, mode="logits"))
#methods.append(IntegratedGradients(model=model, steps=50, mode="probs"))
#methods.append(IntegratedGradients(model=model, steps=50, baseline=0, cc_transform="mean"))


for i, (img, real_label) in enumerate(show_loader):
    if i == 4:
        print(dct[real_label])
        img = prepare(img)
        maps = compare_methods(methods, 
                               model=model, 
                               img=img, 
                               eval_class=None, 
                               img_trafo=transform, 
                               blur=0,
                               viz=viz)
        break